### Chi square test

In [8]:
import pandas as pd

In [9]:
df1 = pd.read_excel(r'D:\Clg material\Desktop\Clg Activities\Projects\Project 2 (LPG&Biogas)\Sorted Data.xlsx',sheet_name = 7)
print(df1.head)
df1.columns

<bound method NDFrame.head of         Primary_fuel  \
0    LPG / Cylinder    
1              Both    
2    LPG / Cylinder    
3              Both    
4    LPG / Cylinder    
..               ...   
241            Both    
242            Both    
243            Both    
244            Both    
245            Both    

    Any_significant_changes_in_LPG_prices_affecting_household_budget  
0                                                 Yes                 
1                                                 Yes                 
2                                                 Yes                 
3                                                   No                
4                                                 Yes                 
..                                                 ...                
241                                                 No                
242                                                 No                
243                                       

Index(['Primary_fuel', 'Any_significant_changes_in_LPG_prices_affecting_household_budget'], dtype='object')

In [10]:
contingency_table = pd.crosstab(df1['Primary_fuel'], df1['Any_significant_changes_in_LPG_prices_affecting_household_budget'])
print('Contingency_table ; ')
print(contingency_table)

Contingency_table ; 
Any_significant_changes_in_LPG_prices_affecting_household_budget  No  Yes 
Primary_fuel                                                              
Both                                                              62    54
LPG / Cylinder                                                    23   107


In [11]:
from scipy.stats import chi2_contingency

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)
print("Chisq statistics = ",chi2_stat)
print("P_value = ",p_value)
print("Degrees_of_freedom = ",dof)

if p_value < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

Chisq statistics =  33.093702692932794
P_value =  8.782269766757862e-09
Degrees_of_freedom =  1
Reject Ho


In [14]:
table = pd.crosstab(D['Primary_fuel'], D['have_cattles'])
print('Contingency_table ; ')
print(table)

Contingency_table ; 
have_cattles     No  Yes
Primary_fuel            
Bio-gas           0   11
Both              0  116
LPG / Cylinder   90   40


### Kruakal Wallis tets

In [15]:
d_f = pd.read_excel(r'D:\Clg material\Desktop\Clg Activities\Projects\Project 2 (LPG&Biogas)\Sorted Data.xlsx',sheet_name = 6)
d_f.head()

,Annual_expenditure_on_biogas,Annual_expenditure_on_LPG,Annual_expenditure_on_both_fules
0,500.0,10000,1000.0
1,200.0,4000,4000.0
2,0.0,3000,2000.0
3,400.0,6000,3000.0
4,2000.0,2000,6000.0


In [16]:
from scipy.stats import kruskal

biogas_exp = d_f['Annual_expenditure_on_biogas'].dropna()    # Drop NaN values if any
lpg_exp = d_f['Annual_expenditure_on_LPG'].dropna()
both_exp = d_f['Annual_expenditure_on_both_fules'].dropna()
print(biogas_exp, lpg_exp, both_exp)

# Perform Kruskal-Wallis test
h_statistic, p_value = kruskal(biogas_exp, lpg_exp, both_exp)

print('H_statistic = ',h_statistic)
print('P_value = ',p_value)

if p_value < 0.05:
    print('Reject Ho i.e. At least one groups median is different from the others.')
else:
    print('Accept Ho i.e. There is no significant difference in the medians of the groups.')

0      500.0
1      200.0
2        0.0
3      400.0
4     2000.0
5     4000.0
6       50.0
7       50.0
8      150.0
9       50.0
10    2000.0
Name: Annual_expenditure_on_biogas, dtype: float64 0      10000
1       4000
2       3000
3       6000
4       2000
       ...  
125    10000
126     9000
127    11500
128    11000
129     9000
Name: Annual_expenditure_on_LPG, Length: 130, dtype: int64 0      1000.0
1      4000.0
2      2000.0
3      3000.0
4      6000.0
        ...  
111    1000.0
112    2000.0
113    2000.0
114    2300.0
115    1000.0
Name: Annual_expenditure_on_both_fules, Length: 116, dtype: float64
H_statistic =  135.74078342174923
P_value =  3.343977875247248e-30
Reject Ho i.e. At least one groups median is different from the others.


### Regression model

In [20]:
import pandas as pd
DT = pd.read_excel(r'D:\Clg material\Desktop\Clg Activities\Projects\Project 2 (LPG&Biogas)\Sorted Data.xlsx',sheet_name = 15)
print(DT.head())
print(DT.dtypes)

   Family_members\n Income_source\n  Annual_income Primary_fuel  Annual_exp
0               4.0               1       100000.0     Bio-gas        500.0
1               3.0               1        40000.0     Bio-gas        200.0
2               4.0               1        30000.0     Bio-gas          0.0
3               5.0               1        40000.0     Bio-gas        400.0
4               5.0               1        40000.0     Bio-gas       2000.0
Family_members\n    float64
Income_source\n      object
Annual_income       float64
Primary_fuel         object
Annual_exp          float64
dtype: object


In [21]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
x = DT['Annual_income'].values.reshape(-1,1)
x = list(scalar.fit_transform(x)) 

In [22]:
# Specify the independent variables (features) and the dependent variable
X = DT[['Family_members\n','Primary_fuel']] # Features
X_additional = pd.DataFrame(x) 
y = DT['Annual_exp']  # Target variable

# Create dummy variables for categorical variable 'Gender'
X_features = pd.get_dummies(X, columns=['Primary_fuel'], drop_first=True)
X = pd.concat([X_features, X_additional], axis=1)
X.fillna(0, inplace=True)

X = X.astype(float)  # Convert all variables to float data type
print(X)
# Check if there are still any missing values
print(X.isnull().sum())

# Add a constant term to the independent variables matrix
X = sm.add_constant(X)

# Fit the multiple linear regression model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

     Family_members\n  Primary_fuel_Both   Primary_fuel_LPG / Cylinder   \
0                 4.0                 0.0                           0.0   
1                 3.0                 0.0                           0.0   
2                 4.0                 0.0                           0.0   
3                 5.0                 0.0                           0.0   
4                 5.0                 0.0                           0.0   
..                ...                 ...                           ...   
253               4.0                 0.0                           1.0   
254               4.0                 0.0                           1.0   
255               5.0                 0.0                           1.0   
256               5.0                 0.0                           1.0   
257               0.0                 0.0                           0.0   

            0  
0   -0.593493  
1   -1.020223  
2   -1.091345  
3   -1.020223  
4   -1.020223  
.. 

In [23]:
# Get predicted y values
predicted_y = model.predict(X)

# Print the predicted y values
print(predicted_y)

0      1241.905840
1      1180.166759
2      1072.165417
3      1012.661338
4      1012.661338
          ...     
253    8775.090920
254    8629.599129
255    8715.586842
256    8836.830001
257    1779.265383
Length: 258, dtype: float64


In [24]:
mean_value = sum(predicted_y)/len(predicted_y)
mean_value

5424.124513618675